# Implementing balancing power constraints

## Objective
In this example, we will implement balancing power constraints in PyPSA in a simplified manner. The objective is to enforce some generators to provide balanacing power by running below their nominal capacity but still keep a linear programming problem formulation. 

To do so, we need to implement additional variables and additional constraints to the model.

## Methodology
We follow the approach that has been presented by Andreas Hösl et al here: https://www.youtube.com/watch?v=fmwDxNpSMM4&t=8043s

The basic idea is that each generator needs to provide reserve capacity **symmetrically**. That means it needs to be able to increase and decrease its output by the same amount in order to contribute to satisfy reserve requirements. This ensures that generators need to operate at partial load in order to provide reserve capacity. 

The following modifications need to be added to the linopy model in a PyPSA Network:

- a new variable $p_{\text{reserve}}(g,t)$ that represents the reserve power provided by generator $g$ in time step $t$. 

- a constraint that ensures that for each time step $t$, the sum of all reserve power provided is greater or equal the required reserves.
$$
\forall t: \sum_{g} p_{\text{reserve}}(g,t) \geq \text{reserve requirement}
$$

- a constraint to ensure that the reserve power a generator provides must be less or equal than the difference between its output $p$ and its nominal capacity $p_\text{nom}$, multiplied with a scalar coefficient $a$. This coefficient can have any value between 0 and 1 and represents the technical availability of a generator to provide balancing power. 
$$
\forall g, t: p_\text{reserve}(g, t) \leq a(g) p_\text{nom}(g) - p(g,t)
$$

- a constraint  to ensure that the balancing power a generator provides must be less or equal than its actual output $p$, multiplied with a scalar coefficient $b$. This coefficient can have any value between 0 and 1 and represents the technical availability of a generator to provide balancing power. 

$$
\forall g, t: p_\text{reserve}(g, t) \leq b(g) p(g,t)
$$

The relationships between the variables $a$, $b$, $p_\text{nom}$, $p$, and $p_\text{reserve}$ are depicted in the following schematic graph.

![balancing-reserves-graph](../../doc/img/balancing-reserves-graph.png)

## Limitations and other approaches

Note that this is a highly simplified approach that has significant limitations:
- It does not distinguish between different categories of reserve power (primary, secondary and xx reserves). 
- Reserves are provided symmetrical; there is no distinction between positive and negative reserves
- The approach only takes into account the provision of balancing power, but not the actual call for balancing power

All these issues can be taken into account in a MIP unit commitment model, albeit at much higher numerical costs. 

Also note that reserve margin constraints have been added to PyPSA-EUR: https://github.com/PyPSA/pypsa-eur/blob/662492a23e4b0fe84f8d65611aad6668488aa88c/scripts/solve_network.py#L393-L472


## Implementation
We start by importing the required packages:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pypsa

### The basic model

Our toy model consits of a single bus with generators that have different marginal costs. We use a sine function for the load profile.

In [ ]:
def setup_network():
    n = pypsa.Network()

    n.add("Carrier", name="carrier1")

    n.add("Bus", name="bus1", carrier="carrier1")

    # add generators with increasing marginal cost
    n.add("Generator", name="gen1", bus="bus1", p_nom=10, marginal_cost=1)
    n.add("Generator", name="gen2", bus="bus1", p_nom=10, marginal_cost=2)
    n.add("Generator", name="gen3", bus="bus1", p_nom=10, marginal_cost=3)
    n.add("Generator", name="gen4", bus="bus1", p_nom=10, marginal_cost=4)
    n.add("Generator", name="gen5", bus="bus1", p_nom=10, marginal_cost=5)
    n.add("Generator", name="gen6", bus="bus1", p_nom=10, marginal_cost=6)
    n.add("Generator", name="gen7", bus="bus1", p_nom=10, marginal_cost=7)
    n.add("Generator", name="gen8", bus="bus1", p_nom=100, marginal_cost=8)

    # create 48 snapshots
    snapshots = np.arange(1, 49)
    n.set_snapshots(snapshots)

    # create load and add to Network
    load_max = 60
    load_profile = np.sin(snapshots / 12 * np.pi) + 3.5
    load_profile = load_profile / load_profile.max() * load_max
    n.add("Load", name="load1", bus="bus1", p_set=load_profile)
    return n

As a reference point, we create the model without any additional constraints and solve it:


In [ ]:
n = setup_network()
n.optimize()

We plot the dispatch over time. As expected, the generators are dispatched strictly according to their marginal cost, each one running at nominal capacity until demand is met. 

In [ ]:
n.generators_t["p"].plot.area().legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))
plt.show()

### Adding custum variable and constraints:

Now let's modify the model by adding some additional constraints! We create a new network and create a model instance attached to it.  Now we can inspect the model instance to get a list of variables and constraints: 

In [ ]:
n_mod = setup_network()
n_mod.optimize.create_model()
n_mod.model

We now add a new variable ``p_reserve`` that represents reserve power. It has a lower bound of zero, and it is defined for all dispatchable generators and has a time index.

Note that PyPSA requires the name of the associated component class to be prepended to the variable or constraint name so that the solution is correctly written to the result network (see https://github.com/PyPSA/PyPSA/issues/664). In this case, the name of the new variable needs to start with ``Generator-``.

In [ ]:
v_rp = n_mod.model.add_variables(
    lower=0,
    coords=[n_mod.snapshots, n_mod.generators.index],
    name="Generator-p_reserve",
)
v_rp

Next, we define a new constraint that ensures that for each snapshot, total reserve requirement is met by the sum of the reserve power provided by all generators.


In [ ]:
reserve_req = 20

c_sum = n_mod.model.add_constraints(
    v_rp.sum("Generator") >= reserve_req, name="GlobalConstraint-sum_of_reserves"
)
c_sum



Now we need to limit the amount of balancing power that each generator can provide. The following  constraint ensures that the balancing power a generator provides must be less or equal than the difference between its output ``p`` and its nominal capacity ``p_nom``:

In [ ]:
a = 1

c_rpos = n_mod.model.add_constraints(
    v_rp <= -n_mod.model.variables["Generator-p"] + a * n_mod.generators["p_nom"],
    name="Generator-reserve_upper_limit",
)
c_rpos

And, lastly, we add a constraint to ensure that the balancing power a generator provides must be less or equal than its actual output ``p``, multiplied with a scalar coefficient ``b``. This coefficient can have any value between 0 and 1 and represents the technical availability of a generator to provide balancing power. 

In [ ]:
b = 0.7

c_rneg = n_mod.model.add_constraints(
    v_rp <= b * n_mod.model.variables["Generator-p"],
    name="Generator-reserve_lower_limit",
)
c_rneg

We can now inspect the model formulation and see that our new variables and constraints have been successfully added:

In [ ]:
n_mod.model

We can now solve the modified model:

In [ ]:
n_mod.optimize.solve_model()

### Examine results

Now we can create a plot to examine the results. 

- The reserve requirement of 20 MW is satisfied in each time step.
- To be able to provide reserves, some generators always need to run below their nominal capacity.
- Of all running generators, those with the highest marginal cost provide as much reserve capacity as possible. 
 

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(10, 5))
n_mod.generators_t["p"].plot.area(ax=axs[0], title="p", legend=False)
n_mod.generators_t["p_reserve"].plot.area(ax=axs[1], title="p_reserve")
plt.tight_layout()
plt.show()

### Comparing both model versions:

To be able to better compare the model versions with and without balancing constraints, we can encapsulate the functionality of adding the new variable and constraints to a function: 

In [ ]:
def add_balancing_constraints(
    n: pypsa.Network, reserve_req: float = 0, a: float = 1, b: float = 1
) -> pypsa.Network:
    """
    Add balancing constraints to network.

    - reserve_req (float): reserve requirement (MW)
    - a (float): scaling coefficient for technical ability to provide reserve capacities. Should be between 0 and 1.
    - b (float): scaling coefficient for technical ability to provide reserve capacities. Should be between 0 and 1.
    """

    # create model instance:
    n.optimize.create_model()

    # add new variable reserves:
    v_rp = n.model.add_variables(
        lower=0, coords=[n.snapshots, n.generators.index], name="Generator-p_reserve"
    )

    # add constraint (reserve requirements to be met in each timestep):
    n.model.add_constraints(
        v_rp.sum("Generator") >= reserve_req, name="GlobalConstraint-sum_of_reserves"
    )

    # add constraint (reserve must be less than diff between p and p_nom)
    n.model.add_constraints(
        v_rp <= -n.model.variables["Generator-p"] + a * n.generators["p_nom"],
        name="Generator-reserve_upper_limit",
    )

    # add constraint (reserve must be larger than b*p)
    n.model.add_constraints(
        v_rp <= b * n.model.variables["Generator-p"],
        name="Generator-reserve_lower_limit",
    )
    return n

Now we can run the model twice, once with and once without balancing constraints, repeat the steps above for each variant, and compare the results:

In [ ]:
n1 = setup_network()
add_balancing_constraints(n1, reserve_req=0, a=0.8, b=0.7)
n1.optimize.solve_model()

n2 = setup_network()
add_balancing_constraints(n2, reserve_req=20, a=0.8, b=0.7)
n2.optimize.solve_model()

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 8))
for i, (n, r) in enumerate(zip([n1, n2], [0, 20])):
    n.generators_t["p"].plot.area(
        ax=axs[0, i], ylabel="p [MW]", title=f"{r} MW reserve required", legend=False
    )
    n.generators_t["p_reserve"].plot.area(
        ax=axs[1, i], ylabel="p_reserve [MW]", legend=i == 1
    )

plt.tight_layout()
plt.show()

We can also compare the average power and reserve power provision over time. The plot shows that adding balancing constraints reduces the average power of the cheaper generators and increases the output of the more expensive ones. 

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(10, 4))
for i, param in enumerate(["p", "p_reserve"]):
    data = pd.concat(
        [n.generators_t[param].mean() for n in [n1, n2]], axis=1, keys=["0 MW", "20 MW"]
    )
    data.plot(ax=axs[i], kind="bar", title=param, legend=i == 1)
plt.show()

And that's it. Feel free to change to values for ``reserve_req``, ``a`` and ``b`` and see how this affects the results. Be careful however that too large reserve requirements can render the model infeasible.

You can also try out an interactive dashboard to play around with a small example model where you can try out different reserve settings at https://pypsa-reserves-dashboard.streamlit.app/ 
